# Meta programming for ASP with clingo in Python

This is an example to show how you can do meta programming for ASP with [clingo](https://potassco.org/clingo/) in Python.


## Meta programming

*Meta programming* refers to the method of (1) translating the rules of a program (the *object program*) to a representation as facts (this process is called *reification*), which then (2) can be used as input for another program (the *meta program*).

Let's illustrate this with an example. In this example, we will write a meta program in ASP whose answer sets correspond to the answer sets of the original program where an even number of (shown) atoms is true.

## Setting up

Let's import clingo and corresponding reification tools.

In [1]:
import clingo
from clingox.reify import reify_program

## An example object program

Let's take a simple object program, that we will feed into the reification to be used with the meta program (that we will construct below).

The following program has 6 answer sets, when projected to the atoms `a`, `b`, `c`, `d` and `e`.

In [2]:
object_program = """
    { c; d }.
    a :- b.
    b :- #sum { 1,c:c; 1,d:d } >= 2.
    e ; f :- d.
    
    #show a/0.
    #show b/0.
    #show c/0.
    #show d/0.
    #show e/0.
"""

For the sake of completeness, let's inspect the (projected) answer sets of this program.

In [3]:
control = clingo.Control([
    '--project',
])
control.add("base", [], object_program)
control.ground([("base", [])])

control.configuration.solve.models = 0

with control.solve(yield_=True) as handle:
    for answer_set in handle:
        atoms = list(answer_set.symbols(shown=True))
        as_list = [
            str(atom) for atom in atoms
        ]
        as_list.sort()
        as_str = ", ".join(as_list)
        print(f"Answer set: {{ {as_str} }}")

Answer set: { a, b, c, d, e }
Answer set: { d, e }
Answer set: { a, b, c, d }
Answer set: { c }
Answer set: {  }
Answer set: { d }


## Reification

Let's now *reify* the object program, yielding a reified program where
all rules are represented as facts.

Note that reification first triggers a translation of the program to a normal form, where each head is either a disjunctive head or a choice head, and each body is either a normal body or a sum body. Normal heads are special cases of disjunctive heads (with a disjunction of exactly one atom).

In [4]:
reified_symbols = reify_program(
    object_program,
    calculate_sccs=False,
)
reified_program = "".join([
    f"{symbol}.\n"
    for symbol in reified_symbols
])

print(reified_program)

tag(incremental).
atom_tuple(0).
atom_tuple(0,1).
atom_tuple(0,2).
literal_tuple(0).
rule(choice(0),normal(0)).
atom_tuple(1).
atom_tuple(1,3).
atom_tuple(1,4).
literal_tuple(1).
literal_tuple(1,2).
rule(disjunction(1),normal(1)).
atom_tuple(2).
atom_tuple(2,5).
weighted_literal_tuple(0).
weighted_literal_tuple(0,1,1).
weighted_literal_tuple(0,2,1).
rule(disjunction(2),sum(0,2)).
atom_tuple(3).
atom_tuple(3,6).
literal_tuple(2).
literal_tuple(2,5).
rule(disjunction(3),normal(2)).
atom_tuple(4).
atom_tuple(4,7).
literal_tuple(3).
literal_tuple(3,6).
rule(disjunction(4),normal(3)).
literal_tuple(4).
literal_tuple(4,7).
output(a,4).
output(b,3).
literal_tuple(5).
literal_tuple(5,1).
output(c,5).
output(d,1).
literal_tuple(6).
literal_tuple(6,3).
output(e,6).



### The reification format

The reified representation of the rules of a program as facts works as follows:
- Atoms are represented using positive integers (`1`, `2`, etc).
- Literals are represented using (positive or negative) integers: positive integers represent positive literals, and negative integers represent negative literals.
- Rules are represented using facts of the form `rule(H,B)`, where `H` indicates the head of the rule and `B` indicates the body of the rule.
- A disjunctive head is represented using the term `disjunction(T)`, where `T` is the index of an 'atom tuple' containing all the atoms in the head.
- A choice head is represented using the term `choice(T)`, where `T` is the index of an 'atom tuple' containing all the atoms in the head.
- A normal body is represented using the term `normal(T)`, where `T` is the index of a 'literal tuple' containing all the literals in the body.
- A sum body is represented using the term `sum(T,K)`, where `T` is the index of a 'weighted literal tuple' containing all the weighted literals in the body, and where `K` is the minimal value that the sum should evaluate to to make the sum body true.

- 'Atom tuples' are declared using the fact `atom_tuple(T)`, where `T` is the index of the atom tuple. The elements of the tuple are declared using facts `atom_tuple(T,A)`, indicating that the atom `A` is part of atom tuple `T`.
- 'Literal tuples' are declared using the fact `literal_tuple(T)`, where `T` is the index of the literal tuple. The elements of the tuple are declared using facts `literal_tuple(T,L)`, indicating that the literal `L` is part of literal tuple `T`.
- 'Weighted literal tuples' are declared using the fact `weighted_literal_tuple(T)`, where `T` is the index of the weighted literal tuple. The elements of the tuple are declared using facts `weighted_literal_tuple(T,L,V)`, indicating that the literal `L` with value `V` is part of weighted literal tuple `T`.

- Facts of the form `output(name,A)` indicate that the atom `A` is to be shown in the (projected) answer set using name `name`.

## The meta program

Let's now write our meta program, whose answer sets correspond to the answer sets of the object program, with the restriction that only an even number of (shown) atoms is true.

In [5]:
meta_program = """
    % Express when a literal tuple B is true, using conjunction/1
    conjunction(B) :- literal_tuple(B),
            hold(L) : literal_tuple(B, L), L > 0;
        not hold(L) : literal_tuple(B,-L), L > 0.

    % Express when a body is true, using body/1
    body(normal(B)) :- rule(_,normal(B)), conjunction(B).
    body(sum(B,G))  :- rule(_,sum(B,G)),
        #sum { W,L :     hold(L), weighted_literal_tuple(B, L,W), L > 0 ;
               W,L : not hold(L), weighted_literal_tuple(B,-L,W), L > 0 } >= G.

    % Express the effect of rules, if the body is true
      hold(A) : atom_tuple(H,A)   :- rule(disjunction(H),B), body(B).
    { hold(A) : atom_tuple(H,A) } :- rule(     choice(H),B), body(B).

    % Count the number of true (shown) atoms, and require this to be even
    num_shown_true_atoms(N) :-
        N = #count { B : output(T,B), conjunction(B) }.
    :- num_shown_true_atoms(N), N \ 2 == 1.

    % Show only the (shown) atoms of the object program
    #show.
    #show model(T) : output(T,B), conjunction(B).
"""

## Combining the meta program and the reified program
Now that we have a reified version of the object program, and a meta program, we can simply call clingo on the union of these.

The resulting answer sets then correspond to the answer sets of the object program that make an even number of (shown) atoms true.

Let's try it out!

In [6]:
# Load and ground reified program, together with meta programs
control = clingo.Control([
    '--project',
    '--warn=none',
])
control.add("base", [], reified_program)
control.add("base", [], meta_program)
control.ground([("base", [])])

control.configuration.solve.models = 0

with control.solve(yield_=True) as handle:
    for answer_set in handle:
        atoms = list(answer_set.symbols(shown=True))
        as_list = [
            str(atom.arguments[0]) for atom in atoms
            if atom.name == "model"
        ]
        as_list.sort()
        as_str = ", ".join(as_list)
        print(f"Answer set: {{ {as_str} }}")

Answer set: { a, b, c, d }
Answer set: {  }
Answer set: { d, e }
